In [176]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [177]:
df1 = pd.read_csv("TrainingData_V1.csv")
df2 = pd.read_excel("TestingData_For_Candidate.xlsx", sheet_name= "Sheet1")

In [178]:
df1["train_test_indicator"]="train"
df2["train_test_indicator"]="test"

In [179]:
df=df1.append(df2, ignore_index = True)

In [180]:
#df.to_csv("Combined_Data.csv")

In [181]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,train_test_indicator
0,1,22-06-2016,27-06-2016,643,38,navy,30,49.9,30822,Mrs,17-04-1969,1013,23-06-2016,0.0,train
1,10,22-06-2016,27-06-2016,195,xxl,grey,46,19.9,30823,Mrs,22-04-1970,1001,15-03-2015,1.0,train
2,11,22-06-2016,05-07-2016,25,xxl,grey,5,79.9,30823,Mrs,22-04-1970,1001,15-03-2015,0.0,train
3,32,23-06-2016,26-06-2016,173,m,brown,20,19.9,17234,Mrs,09-01-1960,1013,17-02-2015,0.0,train
4,43,23-06-2016,26-06-2016,394,40,black,44,90.0,30827,Mrs,NaN,1006,09-02-2016,1.0,train


In [182]:
for col in ['order_date','delivery_date','user_dob','user_reg_date']:
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

In [183]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,train_test_indicator
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,1969-04-17,1013,2016-06-23,0.0,train
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,1970-04-22,1001,2015-03-15,1.0,train
2,11,2016-06-22,2016-05-07,25,xxl,grey,5,79.9,30823,Mrs,1970-04-22,1001,2015-03-15,0.0,train
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,1960-09-01,1013,2015-02-17,0.0,train
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,NaT,1006,2016-09-02,1.0,train


In [184]:
df['Delivery_days']= (df['delivery_date'] - df['order_date'])

In [185]:
df['User_Age']= np.trunc((df['order_date'] - df['user_dob'])/np.timedelta64(1,'Y'))

In [186]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return,train_test_indicator,Delivery_days,User_Age
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,1969-04-17,1013,2016-06-23,0.0,train,5 days,47.0
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,1970-04-22,1001,2015-03-15,1.0,train,5 days,46.0
2,11,2016-06-22,2016-05-07,25,xxl,grey,5,79.9,30823,Mrs,1970-04-22,1001,2015-03-15,0.0,train,-46 days,46.0
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,1960-09-01,1013,2015-02-17,0.0,train,3 days,55.0
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,NaT,1006,2016-09-02,1.0,train,3 days,NaN


In [187]:
df = df.dropna(how='any', subset=['delivery_date'])

In [ ]:
df=df[df['Delivery_days']>=0]

In [ ]:
# df= df[df['Delivery_days']>0]

In [188]:
df_train=df[df['train_test_indicator']=="train"]
df_test=df[df['train_test_indicator']=="test"]

In [189]:
print("train",df_train.shape)
print("test",df_test.shape)

train (72509, 17)
test (18173, 17)


In [190]:
df_test=df_test[df_test['item_id'].isin(df_train['item_id'].unique())]

In [191]:
df_test.shape

(18122, 17)

In [139]:
print(min(df_train['User_Age']))
print(max(df_train['User_Age']))

4.0
116.0


In [140]:
LL=df_train['User_Age'].mean() - 3*df_train['User_Age'].std()
UL=df_train['User_Age'].mean() + 3*df_train['User_Age'].std()
print("Lowest allowed",LL)
print("Highest allowed",UL)
((df_train['User_Age'] < LL) | (df_train['User_Age'] > UL)).sum()

Lowest allowed 17.90809235851213
Highest allowed 85.58551424346484


925

In [141]:
#CAPPING
upper_limit = df_train['User_Age'].mean() + 3*df_train['User_Age'].std()
lower_limit = df_train['User_Age'].mean() - 3*df_train['User_Age'].std()

df_train['User_Age'] = np.where(df_train['User_Age']>upper_limit,upper_limit,
                 np.where(df_train['User_Age']<lower_limit,lower_limit, df_train['User_Age']))

<ipython-input-141-891ea03ab44e>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['User_Age'] = np.where(df_train['User_Age']>upper_limit,upper_limit,


In [142]:
print(min(df_train['User_Age']))
print(max(df_train['User_Age']))

17.90809235851213
85.58551424346484


In [136]:
# Q1 = df_train['User_Age'].quantile(0.25)
# Q3 = df_train['User_Age'].quantile(0.75)
# IQR = Q3 - Q1
# LL= Q1 - 1.5 * IQR
# UL= Q3 + 1.5 * IQR
# print("Lowest allowed",LL)
# print("Highest allowed",UL)
# ((df_train['User_Age'] < LL) | (df_train['User_Age'] > UL)).sum()
# Lowest allowed 28.5
# Highest allowed 72.5
# 1833

Lowest allowed 29.5
Highest allowed 73.5


1849

In [ ]:
upper_limit = df_test['User_Age'].mean() + 3*df_test['User_Age'].std()
lower_limit = df_test['User_Age'].mean() - 3*df_test['User_Age'].std()

df_test['User_Age'] = np.where(df_test['User_Age']>upper_limit,upper_limit,
                 np.where(df_test['User_Age']<lower_limit,lower_limit, df_test['User_Age']))

In [143]:
df_train['User_Age'].isnull().sum()

6347

In [145]:
df_train.groupby('user_title')['User_Age'].mean()

user_title
Company         51.353535
Family          47.852291
Mr              51.153981
Mrs             51.376999
not reported    54.176471
Name: User_Age, dtype: float64

In [146]:
df_train['User_Age'] = df_train['User_Age'].fillna(df_train.groupby('user_title')['User_Age'].transform('mean'))
df_train['User_Age'].isnull().sum()

<ipython-input-146-91c8089a5d35>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['User_Age'] = df_train['User_Age'].fillna(df_train.groupby('user_title')['User_Age'].transform('mean'))


0

In [ ]:
df_test['User_Age'] = df_test['User_Age'].fillna(df_test.groupby('user_title')['User_Age'].transform('mean'))
df_test['User_Age'].isnull().sum()

In [150]:
df_train['item_color'].unique()

array(['navy', 'grey', 'brown', 'black', 'bordeaux', 'white', 'purple',
       'magenta', 'stained', 'blue', 'red', 'olive', 'ocher', 'ash',
       'khaki', 'nature', 'denim', 'curry', 'beige', 'turquoise', 'green',
       'anthracite', 'yellow', 'berry', 'petrol', 'brwon', 'dark denim',
       'hibiscus', 'azure', 'ecru', 'gold', 'orange', 'silver',
       'darkblue', 'mocca', 'mint', 'pink', 'copper coin', 'jade', 'blau',
       'aqua', 'cobalt blue', 'mango', 'aviator', 'champagner',
       'aubergine', 'cognac', 'fuchsia', 'pallid', 'aquamarine',
       'terracotta', 'ancient', 'curled', 'apricot', 'coral', 'basalt',
       'floral', 'creme', 'mahagoni', 'dark garnet', 'striped', 'ivory',
       nan, 'dark oliv', 'dark navy', 'habana', 'antique pink',
       'dark grey', 'amethyst', 'currant purple', 'kanel', 'ebony',
       'avocado', 'caramel', 'baltic blue', 'almond', 'opal'],
      dtype=object)

In [151]:
df_train['item_color'].nunique()

76

In [ ]:
df_train['item_color']=df_train['item_color'].replace("?",np.NaN)
df_test['item_color']=df_test['item_color'].replace("?",np.NaN)

In [152]:
print("train_null",df_train['item_color'].isnull().sum())
print("test_null",df_test['item_color'].isnull().sum())

76

In [153]:
df_train['item_color'].fillna(df_train['item_color'].mode()[0], inplace=True)
df_test['item_color'].fillna(df_test['item_color'].mode()[0], inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [ ]:
df_train['item_color'].value_counts()

In [209]:
for enc_vars in ['item_id','item_size','item_color','brand_id']:
    mean_encode=df_train.groupby(enc_vars)['return'].mean()
    print(mean_encode)
    df_train.loc[:,enc_vars+"_enc"]=df_train[enc_vars].map(mean_encode)
    df_test.loc[:,enc_vars+"_enc"]=df_test[enc_vars].map(mean_encode)

item_id
1       0.486486
2       0.469388
3       0.362069
4       0.611650
5       0.538835
          ...   
2234    1.000000
2237    0.000000
2238    0.000000
2240    1.000000
2241    0.500000
Name: return, Length: 1879, dtype: float64
item_size
1          0.571429
10         0.601990
10+        0.527559
100        0.500000
104        0.225806
             ...   
unsized    0.304325
xl         0.496041
xs         0.450000
xxl        0.467988
xxxl       0.388889
Name: return, Length: 100, dtype: float64
item_color
?             0.062500
almond        0.166667
amethyst      0.000000
ancient       0.434783
anthracite    0.517934
                ...   
striped       0.561224
terracotta    0.571429
turquoise     0.496195
white         0.485876
yellow        0.381395
Name: return, Length: 77, dtype: float64
brand_id
1      0.550867
2      0.455000
3      0.397829
4      0.553846
5      0.468722
         ...   
134    0.000000
135    0.000000
136    0.200000
137    0.000000
138    0.000000


In [210]:
df_test.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_state,user_reg_date,return,train_test_indicator,Delivery_days,User_Age,item_id_enc,item_size_enc,item_color_enc,brand_id_enc
79945,26,2016-06-23,2016-06-26,92,xl,turquoise,42,69.90,9392,Mrs,...,1010,2016-03-24,NaN,test,3 days,54.0,0.326087,0.496041,0.496195,0.487509
79946,28,2016-06-23,2016-07-02,2,xxl,green,2,19.90,9392,Mrs,...,1010,2016-03-24,NaN,test,9 days,54.0,0.469388,0.467988,0.475378,0.455000
79947,37,2016-06-23,2016-06-26,895,38,white,39,39.95,30826,Mrs,...,1001,2015-02-17,NaN,test,3 days,52.0,0.585153,NaN,0.485876,0.544348
79950,80,2016-06-23,2016-06-26,57,l,purple,3,24.90,16075,Mrs,...,1003,2015-11-22,NaN,test,3 days,46.0,0.323944,0.491894,0.468941,0.397829
79951,95,2016-06-23,2016-06-26,57,xl,olive,3,34.90,17124,Mrs,...,1015,2015-08-02,NaN,test,3 days,67.0,0.323944,0.496041,0.507324,0.397829


In [225]:
print("test_null",df_test['item_size_enc'].isnull().sum())

test_null 8108


In [212]:
df_train.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,...,user_state,user_reg_date,return,train_test_indicator,Delivery_days,User_Age,item_id_enc,item_size_enc,item_color_enc,brand_id_enc
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.9,30822,Mrs,...,1013,2016-06-23,0.0,train,5 days,47.0,0.235294,0.519857,0.370192,0.397550
1,10,2016-06-22,2016-06-27,195,xxl,grey,46,19.9,30823,Mrs,...,1001,2015-03-15,1.0,train,5 days,46.0,0.475410,0.467988,0.528966,0.514665
2,11,2016-06-22,2016-05-07,25,xxl,grey,5,79.9,30823,Mrs,...,1001,2015-03-15,0.0,train,-46 days,46.0,0.492537,0.467988,0.528966,0.468722
3,32,2016-06-23,2016-06-26,173,m,brown,20,19.9,17234,Mrs,...,1013,2015-02-17,0.0,train,3 days,55.0,0.701107,0.492122,0.534008,0.620771
4,43,2016-06-23,2016-06-26,394,40,black,44,90.0,30827,Mrs,...,1006,2016-09-02,1.0,train,3 days,NaN,0.627451,0.552768,0.515076,0.586207
